# Graphics Data Export

Export visualization data from the `db_noted` database to JSON for interactive charts.

**Purpose**: Generate chart data for admin analytics page with interactive graphs and filters.

# Setup

Import libraries, connect to database, define chart data structure.

In [1]:
# Import libraries for graphics and database
import mysql.connector
import json
from datetime import datetime, timedelta
import pandas as pd
import calendar
import os

print("✅ Graphics libraries imported!")
print(f"📅 Executed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Database connection
try:
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='db_noted'
    )
    cursor = conn.cursor()
    print("✅ Database connection established")
except Exception as e:
    print(f"❌ Connection error: {e}")
    exit()

# Helper function to check connection and reconnect if needed
def ensure_connection():
    global conn, cursor
    try:
        # Check if connection is alive
        conn.ping(reconnect=False, attempts=1, delay=0)
    except:
        print("⚠️ Connection lost. Reconnecting...")
        try:
            # Try to reconnect
            conn = mysql.connector.connect(
                host='localhost',
                user='root',
                password='',
                database='db_noted'
            )
            cursor = conn.cursor()
            print("✅ Reconnection successful")
        except Exception as e:
            print(f"❌ Reconnection failed: {e}")
            exit()

# Helper function for safe query execution
def safe_query(query, params=None):
    try:
        ensure_connection()  # Ensure connection is active
        cursor.execute(query, params)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(f"❌ Query error: {e}")
        print(f"Query: {query}")
        return []

# Calculate date ranges
today = datetime.now().date()
month_start = today.replace(day=1)
year_start = today.replace(month=1, day=1)
last_12_months = today - timedelta(days=365)

print(f"📅 Date ranges:")
print(f"   Today: {today}")
print(f"   Month start: {month_start}")
print(f"   Year start: {year_start}")
print(f"   Last 12 months: {last_12_months}")

# SIMPLIFIED GRAPHICS DATA STRUCTURE (removed payment methods and daily orders)
graphics_data = {
    "last_updated": datetime.now().isoformat(),
    "date_ranges": {
        "Last 7 days": {},
        "Last 30 days": {},
        "Last 3 months": {},
        "Last 6 months": {},
        "Last 12 months": {},
        "All time": {}
    },
    "static_charts": {
        "products_chart": {"labels": [], "data": [], "type": "doughnut", "title": "Products by Category"},
        "top_customers": {"labels": [], "data": [], "type": "horizontal_bar", "title": "Top Customers by Spending"}
    },
    "filters": {
        "date_ranges": ["Last 7 days", "Last 30 days", "Last 3 months", "Last 6 months", "Last 12 months", "All time"],
        "categories": []
    }
}

print("\n🎨 Ready to collect simplified graphics data...")

✅ Graphics libraries imported!
📅 Executed at: 2025-07-06 20:34:20
✅ Database connection established
📅 Date ranges:
   Today: 2025-07-06
   Month start: 2025-07-01
   Year start: 2025-01-01
   Last 12 months: 2024-07-06

🎨 Ready to collect simplified graphics data...


# Sales Chart Data

Monthly sales data for line chart visualization.

In [2]:
# Sales data by different date ranges
date_range_queries = {
    "Last 7 days": "DATE_SUB(NOW(), INTERVAL 7 DAY)",
    "Last 30 days": "DATE_SUB(NOW(), INTERVAL 30 DAY)", 
    "Last 3 months": "DATE_SUB(NOW(), INTERVAL 3 MONTH)",
    "Last 6 months": "DATE_SUB(NOW(), INTERVAL 6 MONTH)",
    "Last 12 months": "DATE_SUB(NOW(), INTERVAL 12 MONTH)",
    "All time": "DATE('2020-01-01')"  # Far enough back to get all data
}

for range_name, date_condition in date_range_queries.items():
    print(f"\n📊 Processing sales data for: {range_name}")
    
    # Sales by month for this range
    query = f"""
    SELECT 
        DATE_FORMAT(order_date, '%Y-%m') as month,
        DATE_FORMAT(order_date, '%M %Y') as month_label,
        SUM(total) as total_sales,
        COUNT(*) as order_count
    FROM orders
    WHERE order_date >= {date_condition}
    GROUP BY DATE_FORMAT(order_date, '%Y-%m'), DATE_FORMAT(order_date, '%M %Y')
    ORDER BY month
    """
    result = safe_query(query)
    
    sales_labels = []
    sales_data = []
    
    for row in result:
        sales_labels.append(row[1])  # Month label
        sales_data.append(float(row[2]))  # Total sales
    
    graphics_data["date_ranges"][range_name]["sales_chart"] = {
        "labels": sales_labels,
        "data": sales_data,
        "type": "line",
        "title": f"Sales Over Time ({range_name})"
    }
    
    print(f"   ✅ Sales: {len(sales_data)} months, Total: {sum(sales_data):.2f}€")

print(f"\n✅ Sales chart data collected for all date ranges")


📊 Processing sales data for: Last 7 days
   ✅ Sales: 2 months, Total: 88.99€

📊 Processing sales data for: Last 30 days
   ✅ Sales: 2 months, Total: 855.98€

📊 Processing sales data for: Last 3 months
   ✅ Sales: 4 months, Total: 2026.96€

📊 Processing sales data for: Last 6 months
   ✅ Sales: 7 months, Total: 3418.92€

📊 Processing sales data for: Last 12 months
   ✅ Sales: 7 months, Total: 3897.92€

📊 Processing sales data for: All time
   ✅ Sales: 7 months, Total: 3897.92€

✅ Sales chart data collected for all date ranges


# Orders Chart Data

Monthly orders count for bar chart visualization.

In [3]:
# Orders chart generation for all date ranges
# --------------------------------------------

# Generate orders data for all date ranges
def generate_orders_data(days_back=None, months_back=None):
    if days_back:
        # Daily data for short ranges
        query = """
        SELECT 
            DATE(order_date) as period,
            DATE_FORMAT(order_date, '%d/%m') as period_label,
            COUNT(*) as order_count
        FROM orders
        WHERE order_date >= DATE_SUB(NOW(), INTERVAL %s DAY)
        GROUP BY DATE(order_date), DATE_FORMAT(order_date, '%d/%m')
        ORDER BY period
        """
        params = (days_back,)
        title_suffix = f"(Last {days_back} days)"
    elif months_back:
        # Monthly data for longer ranges
        query = """
        SELECT 
            DATE_FORMAT(order_date, '%Y-%m') as period,
            DATE_FORMAT(order_date, '%M %Y') as period_label,
            COUNT(*) as order_count
        FROM orders
        WHERE order_date >= DATE_SUB(NOW(), INTERVAL %s MONTH)
        GROUP BY DATE_FORMAT(order_date, '%Y-%m'), DATE_FORMAT(order_date, '%M %Y')
        ORDER BY period
        """
        params = (months_back,)
        title_suffix = f"(Last {months_back} months)"
    else:
        # All time - monthly data
        query = """
        SELECT 
            DATE_FORMAT(order_date, '%Y-%m') as period,
            DATE_FORMAT(order_date, '%M %Y') as period_label,
            COUNT(*) as order_count
        FROM orders
        GROUP BY DATE_FORMAT(order_date, '%Y-%m'), DATE_FORMAT(order_date, '%M %Y')
        ORDER BY period
        """
        params = None
        title_suffix = "(All time)"
    
    result = safe_query(query, params)
    
    labels = []
    data = []
    for row in result:
        labels.append(row[1])  # Period label
        data.append(row[2])    # Order count
    
    return {
        "labels": labels,
        "data": data,
        "type": "bar",
        "title": f"Orders by Period {title_suffix}"
    }

# Generate orders data for all date ranges
graphics_data["date_ranges"]["Last 7 days"]["orders_chart"] = generate_orders_data(days_back=7)
graphics_data["date_ranges"]["Last 30 days"]["orders_chart"] = generate_orders_data(days_back=30)
graphics_data["date_ranges"]["Last 3 months"]["orders_chart"] = generate_orders_data(months_back=3)
graphics_data["date_ranges"]["Last 12 months"]["orders_chart"] = generate_orders_data(months_back=12)
graphics_data["date_ranges"]["All time"]["orders_chart"] = generate_orders_data()

# Orders count by different date ranges
for range_name, date_condition in date_range_queries.items():
    print(f"\n📦 Processing orders data for: {range_name}")
    
    query = f"""
    SELECT 
        DATE_FORMAT(order_date, '%M %Y') as month_label,
        COUNT(*) as order_count
    FROM orders
    WHERE order_date >= {date_condition}
    GROUP BY DATE_FORMAT(order_date, '%Y-%m'), DATE_FORMAT(order_date, '%M %Y')
    ORDER BY DATE_FORMAT(order_date, '%Y-%m')
    """
    result = safe_query(query)
    
    orders_labels = []
    orders_data = []
    
    for row in result:
        orders_labels.append(row[0])
        orders_data.append(row[1])
    
    graphics_data["date_ranges"][range_name]["orders_chart"] = {
        "labels": orders_labels,
        "data": orders_data,
        "type": "bar",
        "title": f"Orders by Month ({range_name})"
    }
    
    print(f"   ✅ Orders: {len(orders_data)} months, Total: {sum(orders_data)}")

print(f"\n✅ Orders charts generated for all date ranges")
for range_name, range_data in graphics_data["date_ranges"].items():
    if "orders_chart" in range_data:
        print(f"   {range_name}: {len(range_data['orders_chart']['data'])} data points")


📦 Processing orders data for: Last 7 days
   ✅ Orders: 2 months, Total: 3

📦 Processing orders data for: Last 30 days
   ✅ Orders: 2 months, Total: 7

📦 Processing orders data for: Last 3 months
   ✅ Orders: 4 months, Total: 13

📦 Processing orders data for: Last 6 months
   ✅ Orders: 7 months, Total: 21

📦 Processing orders data for: Last 12 months
   ✅ Orders: 7 months, Total: 22

📦 Processing orders data for: All time
   ✅ Orders: 7 months, Total: 22

✅ Orders charts generated for all date ranges
   Last 7 days: 2 data points
   Last 30 days: 2 data points
   Last 3 months: 4 data points
   Last 6 months: 7 data points
   Last 12 months: 7 data points
   All time: 7 data points


# Products by Category Chart

Product distribution for doughnut chart visualization.

In [4]:
# Products by category for doughnut chart (static data - doesn't change with date ranges)
query = """
SELECT 
    c.description as category_name,
    COUNT(p.id) as product_count
FROM categories c
LEFT JOIN products p ON c.id = p.category_id
GROUP BY c.id, c.description
HAVING product_count > 0
ORDER BY product_count DESC
"""
result = safe_query(query)

for row in result:
    graphics_data["static_charts"]["products_chart"]["labels"].append(row[0])
    graphics_data["static_charts"]["products_chart"]["data"].append(row[1])
    
    # Add to filters
    if row[0] not in graphics_data["filters"]["categories"]:
        graphics_data["filters"]["categories"].append(row[0])

print(f"✅ Products chart: {len(graphics_data['static_charts']['products_chart']['data'])} categories")
print(f"   Total products: {sum(graphics_data['static_charts']['products_chart']['data'])}")

✅ Products chart: 4 categories
   Total products: 12


# Top Customers Chart

Top 10 customers by total spending for horizontal bar chart.

In [5]:
# Top 10 customers by total spending (static data - doesn't change with date ranges)
query = """
SELECT 
    u.name as customer_name,
    SUM(o.total) as total_spent
FROM users u
JOIN orders o ON u.id = o.user_id
GROUP BY u.id, u.name
ORDER BY total_spent DESC
LIMIT 10
"""
result = safe_query(query)

for row in result:
    graphics_data["static_charts"]["top_customers"]["labels"].append(row[0])
    graphics_data["static_charts"]["top_customers"]["data"].append(float(row[1]))

print(f"✅ Top customers chart: {len(graphics_data['static_charts']['top_customers']['data'])} customers")
print(f"   Total value: {sum(graphics_data['static_charts']['top_customers']['data'])}€")

✅ Top customers chart: 10 customers
   Total value: 3877.93€


# Export to JSON

Save all chart data to graphics.json for frontend visualization.

In [6]:
# Export graphics data to JSONoutput_path = "e:/producao_V15/noted/static/data/graphics.json"
output_path = os.path.join("..", "static", "data", "graphics.json")


def decimal_default(obj):
    """JSON serializer for objects not serializable by default json code"""
    if hasattr(obj, '__float__'):
        return float(obj)
    raise TypeError

# Close database connection
cursor.close()
conn.close()

print(f"\n📊 SIMPLIFIED GRAPHICS DATA SUMMARY:")
print(f"   📅 Date ranges: {len(graphics_data['date_ranges'])}")
for range_name, range_data in graphics_data["date_ranges"].items():
    sales_len = len(range_data.get('sales_chart', {}).get('data', []))
    orders_len = len(range_data.get('orders_chart', {}).get('data', []))
    print(f"     📈 {range_name}: Sales({sales_len}) Orders({orders_len})")

print(f"   🔧 Static charts:")
print(f"     🍩 Products: {len(graphics_data['static_charts']['products_chart']['data'])} categories")
print(f"     👑 Top customers: {len(graphics_data['static_charts']['top_customers']['data'])} customers")
print(f"   🔽 Filter options: {len(graphics_data['filters']['date_ranges'])} date ranges")

# Export to JSON
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(graphics_data, f, indent=2, ensure_ascii=False, default=decimal_default)

print(f"\n✅ Simplified graphics data exported to: {output_path}")

# Verify file size
try:
    with open(output_path, 'r', encoding='utf-8') as f:
        file_content = f.read()
    print(f"📄 JSON file size: {len(file_content)} characters")
except Exception as e:
    print(f"⚠️ Error checking file size: {e}")

# Create backward-compatible default view (Last 12 months)
print(f"\n🔄 Creating backward-compatible default data...")
default_range = graphics_data["date_ranges"]["Last 12 months"]

# Add default data at root level for backward compatibility
if "sales_chart" in default_range:
    graphics_data["sales_chart"] = default_range["sales_chart"]
if "orders_chart" in default_range:
    graphics_data["orders_chart"] = default_range["orders_chart"]

# Add static charts at root level
graphics_data.update(graphics_data["static_charts"])

print("✅ Backward compatibility data added")


📊 SIMPLIFIED GRAPHICS DATA SUMMARY:
   📅 Date ranges: 6
     📈 Last 7 days: Sales(2) Orders(2)
     📈 Last 30 days: Sales(2) Orders(2)
     📈 Last 3 months: Sales(4) Orders(4)
     📈 Last 6 months: Sales(7) Orders(7)
     📈 Last 12 months: Sales(7) Orders(7)
     📈 All time: Sales(7) Orders(7)
   🔧 Static charts:
     🍩 Products: 4 categories
     👑 Top customers: 10 customers
   🔽 Filter options: 6 date ranges

✅ Simplified graphics data exported to: ..\static\data\graphics.json
📄 JSON file size: 5570 characters

🔄 Creating backward-compatible default data...
✅ Backward compatibility data added


# Cleanup

Process completed successfully.

In [7]:
print("\n🎉 Graphics data collection completed!")
print("💡 Use the data with graphics.js:")
print("   fetch('/static/data/graphics.json')")
print("📊 Ready for Chart.js visualization!")


🎉 Graphics data collection completed!
💡 Use the data with graphics.js:
   fetch('/static/data/graphics.json')
📊 Ready for Chart.js visualization!
